In [11]:
#!/usr/bin/env python3
import numpy as np
from ikpy.chain import Chain
from ikpy.link import OriginLink, URDFLink
from dynamixel_sdk import PortHandler, PacketHandler

# ----------------------------
# Define the Kinematic Chain (in meters)
# ----------------------------
# Joint 1: Base rotation (rotates about z)
joint1 = URDFLink(
    name="1_base_rotation",
    origin_translation=np.array([0, 0, 0.080]),  # 80 mm upward from base
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([0, 0, 1]),  
    joint_type="revolute",
    bounds=(0, 3*np.pi/2)
)

# Joint 2: Shoulder tilt
joint2 = URDFLink(
    name="2_shoulder_tilt",
    origin_translation=np.array([0, 0, 0.018]),  
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([0, 1, 0]),
    joint_type="revolute",
    bounds=(-.65, np.pi/2)
)

L2 = 0.095 
joint3 = URDFLink(
    name="3_elbow_tilt",
    origin_translation=np.array([0, 0, L2]),
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([0, 1, 0]),  # Changed to positive y-axis rotation
    joint_type="revolute",
    bounds=(-110 / 180 * np.pi, 1.4)
)


L3 = 0.100  
joint4 = URDFLink(
    name="4_wrist_tilt",
    origin_translation=np.array([L3, 0, 0]),
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([0, -1, 0]), #TODO: is this -1 or 1?
    joint_type="revolute",
    bounds=(-np.pi/2, np.pi/2)
)

L4 = 0.120 #we are saying that 5th one is on 120mm
joint5 = URDFLink(
    name="5_gripper_rotate",
    origin_translation=np.array([0, 0, -L4]),
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([1, 0, 0]),
    joint_type="revolute"
)

# Assemble the robot chain: The first link is the fixed base (OriginLink)
robot_chain = Chain(name="my_robot_arm_upto_joint11", links=[
    OriginLink(),
    joint1,
    joint2,
    joint3,
    joint4,
    joint5
])

# ----------------------------
# Dynamixel Configuration
# ----------------------------
PROTOCOL_VERSION = 2.0
DXL_IDS = [1, 2, 3, 4, 5, 6]  # Use first 5 IDs for joints 7-11.
BAUDRATE = 1e6
DEVICENAME = '/dev/ttyACM0'

TICKS_PER_REV = 4096

def rad_to_dxl(angle_rad):
    angle_rad = angle_rad
    return int((angle_rad / (2 * np.pi)) * TICKS_PER_REV)

def dxl_to_rad(ticks):
    return (ticks / TICKS_PER_REV) * 2 * np.pi

# ----------------------------
# Initialize Port and Packet Handler (No torque enable required here)
# ----------------------------
port_handler = PortHandler(DEVICENAME)
packet_handler = PacketHandler(PROTOCOL_VERSION)
if not port_handler.openPort():
    print("Failed to open port!")
    exit()
if not port_handler.setBaudRate(BAUDRATE):
    print("Failed to set baudrate!")
    exit()

def to_signed(val):
    if val > 0x7FFFFFFF:  # 0x7FFFFFFF is 2^31 - 1
        return val - 0x100000000  # 0x100000000 is 2^32
    return val

# ----------------------------
# Read Current Angles Function
# ----------------------------
def read_current_angles():
    angles = []
    for dxl_id in DXL_IDS:
        pos, dxl_comm_result, dxl_error = packet_handler.read4ByteTxRx(port_handler, dxl_id, 132)
        # print("ID", dxl_id, "TICKS", pos)
        if dxl_comm_result != 0 or dxl_error != 0:
            print(f"Error reading servo {dxl_id}, assuming 0.")
            angles.append(0)
        else:
            angle = dxl_to_rad(to_signed(pos))
            angles.append(angle)
    return np.array(angles)

def forward_kinematics_leader(angles):
    truncated = angles[:5] 
    full_angles = np.concatenate(([0], truncated))  # Prepend dummy for OriginLink
    return robot_chain.forward_kinematics(full_angles)


current_angles = read_current_angles()
print("Current joint angles (deg) from servos:", current_angles*180/ (np.pi))
print("Current joint angles (rad) from servos:", current_angles)

home_pose = forward_kinematics_leader(current_angles)
home_xyz = home_pose[:3, 3] * 100  # Convert from meters to centimeters

print("\nHome position (xyz) of end-effector (joint 11) relative to base (in cm):")
print(f"  x = {home_xyz[0]:.2f} cm")
print(f"  y = {home_xyz[1]:.2f} cm")
print(f"  z = {home_xyz[2]:.2f} cm")



Current joint angles (deg) from servos: [ 13.0078125  -14.765625   -82.70507812  80.06835938 -49.65820312
  -9.75585938]
Current joint angles (rad) from servos: [ 0.22702916 -0.25770877 -1.44347592  1.3974565  -0.86669915 -0.17027187]

Home position (xyz) of end-effector (joint 11) relative to base (in cm):
  x = -3.12 cm
  y = -0.72 cm
  z = 40.89 cm


In [ ]:
import time
import pygame
import collections

sampling_rate = 50  # Hz
dt = 1.0 / sampling_rate

# Duration (in seconds) of data to display on the plot
window_duration = 10.0  # seconds
max_samples = int(window_duration * sampling_rate)

# Create deques to store time and velocity samples
time_data = collections.deque(maxlen=max_samples)
vx_data = collections.deque(maxlen=max_samples)
vy_data = collections.deque(maxlen=max_samples)

# For plotting, set a fixed velocity range (m/s)
velocity_min = -1.0
velocity_max =  1.0

# ----------------------------
# Pygame Setup
# ----------------------------
pygame.init()
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Real-Time Velocity Visualization")
font = pygame.font.SysFont("Arial", 16)

# Define two plot areas: top for Vx, bottom for Vy.
top_rect = pygame.Rect(0, 0, width, height // 2)
bottom_rect = pygame.Rect(0, height // 2, width, height // 2)

# ----------------------------
# Get Initial Sample
# ----------------------------
prev_angles = read_current_angles()
prev_fk = forward_kinematics_leader(prev_angles)
prev_pos = prev_fk[:3, 3]  # Position vector in meters

start_time = time.time()

# ----------------------------
# Main Loop: Sampling + Visualization
# ----------------------------
running = True
while running:
    sample_start = time.time()
    
    # Handle Pygame events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False

    # Sample current state
    current_angles = read_current_angles()
    current_fk = forward_kinematics_leader(current_angles)
    current_pos = current_fk[:3, 3]

    # Compute velocities (m/s) for x and y components
    vx = (current_pos[0] - prev_pos[0]) / dt
    vy = (current_pos[1] - prev_pos[1]) / dt
    prev_pos = current_pos.copy()  # update for next iteration

    # Record the sample with relative time
    current_time = time.time() - start_time
    time_data.append(current_time)
    vx_data.append(vx)
    vy_data.append(vy)

    screen.fill((0, 0, 0))  

    # --------- Top Plot: Vx vs. Time ---------
    pygame.draw.rect(screen, (0, 0, 0), top_rect, 2)  # draw border
    if len(time_data) > 1:
        points = []
        # Plot only data from the last window_duration seconds
        t_min = max(0, current_time - window_duration)
        for t, v in zip(time_data, vx_data):
            # x coordinate scaled to [0, width]
            x = int((t - t_min) / window_duration * width)
            # y coordinate: map velocity_min->velocity_max to bottom->top of top_rect
            v_norm = (v - velocity_min) / (velocity_max - velocity_min)
            y = int(top_rect.bottom - v_norm * top_rect.height)
            points.append((x, y))
        if len(points) >= 2:
            pygame.draw.lines(screen, (255, 0, 0), False, points, 2)
    
    # Draw axis labels and ticks for Vx
    label = font.render("Vx (m/s)", True, (255, 255, 255))
    screen.blit(label, (10, 10))
    
    # Draw velocity ticks on y-axis
    for v in range(int(velocity_min), int(velocity_max) + 1):
        y = int(top_rect.bottom - ((v - velocity_min) / (velocity_max - velocity_min)) * top_rect.height)
        pygame.draw.line(screen, (255, 255, 255), (45, y), (55, y), 1)
        v_label = font.render(f"{v}", True, (255, 255, 255))
        screen.blit(v_label, (20, y - 8))
    
    # --------- Bottom Plot: Vy vs. Time ---------
    pygame.draw.rect(screen, (0, 0, 0), bottom_rect, 2)  # draw border
    if len(time_data) > 1:
        points = []
        t_min = max(0, current_time - window_duration)
        for t, v in zip(time_data, vy_data):
            x = int((t - t_min) / window_duration * width)
            v_norm = (v - velocity_min) / (velocity_max - velocity_min)
            y = int(bottom_rect.bottom - v_norm * bottom_rect.height)
            points.append((x, y))
        if len(points) >= 2:
            pygame.draw.lines(screen, (0, 0, 255), False, points, 2)
    
    # Draw axis labels and ticks for Vy
    label2 = font.render("Vy (m/s)", True, (255, 255, 255))
    screen.blit(label2, (10, bottom_rect.top + 10))
    
    # Draw velocity ticks on y-axis for Vy
    for v in range(int(velocity_min), int(velocity_max) + 1):
        y = int(bottom_rect.bottom - ((v - velocity_min) / (velocity_max - velocity_min)) * bottom_rect.height)
        pygame.draw.line(screen, (255, 255, 255), (45, y), (55, y), 1)
        v_label = font.render(f"{v}", True, (255, 255, 255))
        screen.blit(v_label, (20, y - 8))

    # Draw time axis labels
    time_label = font.render("Time (s)", True, (255, 255, 255))
    screen.blit(time_label, (width - 70, height - 20))
    
    pygame.display.flip()

    # Maintain the 100Hz sampling rate
    elapsed = time.time() - sample_start
    sleep_time = dt - elapsed
    if sleep_time > 0:
        time.sleep(sleep_time)

# Clean up on exit
pygame.quit()
port_handler.closePort()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-02-13 16:54:26.326 python[6566:15268939] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-13 16:54:26.326 python[6566:15268939] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Error reading servo 4, assuming 0.


: 